In [1]:
import pandas as pd
import numpy as np
import os
import sys
import Mixture
import multiprocessing


In [2]:
print('Num cores: ' + str(multiprocessing.cpu_count()))

cores = 4
iters = 4
output = 'NewmanFL_TIL10_500pops'

if cores > multiprocessing.cpu_count():
    cores = multiprocessing.cpu_count()

Num cores: 4


In [3]:
# Read xlsx files
X = pd.read_excel('app/data/TIL10_signature.xlsx', sheet_name = 0) 

In [4]:
# Read xlsx files
Y = pd.read_excel('data/NewmanFL.xlsx', sheet_name = 0) 

In [5]:
# Run Mixer Function
if __name__ == '__main__':
    result, pValues = Mixture.Mixture(X, Y , cores, iters, output)

Running mixer with subjects (Count: 15)...
Normalizing data
Processing...
----------------------------------------------------------------------------------------------------
--------------------------------------------------
Subject: GSM1587833_FL_lymph_node_biopsy_untreated_575.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
Subject: GSM1587831_FL_lymph_node_biopsy_untreated_1063.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
Subject: GSM1587832_FL_lymph_node_biopsy_untreated_1080.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------

--------------------------------------------------
Subject: GSM1587834_FL_lymph_node_biopsy_untreated_581.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
Iter: 1 Subject: GSM1587831_FL_lymph_node_biopsy_untreat

In [8]:
oo = result.Subjects[0].MIXprop[0].to_numpy(copy = True)
oo

array([[0.90256962, 0.09743038, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.7136939 , 0.1289198 , 0.        , 0.        , 0.        ,
        0.        , 0.11802777, 0.03935852, 0.        , 0.        ],
       [0.79589517, 0.09674959, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.03918719, 0.06816806, 0.        ],
       [0.46974898, 0.30925149, 0.        , 0.        , 0.        ,
        0.        , 0.18748422, 0.03351531, 0.        , 0.        ],
       [0.60193307, 0.11220227, 0.16187135, 0.        , 0.        ,
        0.        , 0.10233414, 0.02165916, 0.        , 0.        ],
       [0.49398868, 0.12679016, 0.06592614, 0.        , 0.        ,
        0.        , 0.26288403, 0.05041099, 0.        , 0.        ],
       [0.70989344, 0.14510445, 0.        , 0.        , 0.        ,
        0.        , 0.05838125, 0.05919139, 0.02742947, 0.        ],
       [0.88472318, 0.0553045 , 0.       

In [28]:
a = [result.Subjects[0].MIXprop[0].iloc[j].values for j in range(len(result.Subjects[0].MIXprop[0]))]

In [37]:
np.transpose(a)

array([[0.90256962, 0.7136939 , 0.79589517, 0.46974898, 0.60193307,
        0.49398868, 0.70989344, 0.88472318, 0.89564802, 0.67255126,
        0.56230175, 0.93644457, 0.54997657, 0.66715859],
       [0.09743038, 0.1289198 , 0.09674959, 0.30925149, 0.11220227,
        0.12679016, 0.14510445, 0.0553045 , 0.        , 0.15353534,
        0.08673577, 0.01161619, 0.04595101, 0.13556801],
       [0.        , 0.        , 0.        , 0.        , 0.16187135,
        0.06592614, 0.        , 0.        , 0.        , 0.        ,
        0.20400794, 0.        , 0.06601392, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0. 

In [17]:
result.Subjects[0].MIXprop[0]

,B.cells,Macrophages.M1,Macrophages.M2,Monocytes,Neutrophils,NK.cells,T.cells.CD4,T.cells.CD8,Tregs,Dendritic.cells
GSM1587831_FL_lymph_node_biopsy_untreated_1063.CEL.gz,0.902570,0.097430,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
GSM1587832_FL_lymph_node_biopsy_untreated_1080.CEL.gz,0.713694,0.128920,0.000000,0.0,0.0,0.0,0.118028,0.039359,0.000000,0.0
GSM1587833_FL_lymph_node_biopsy_untreated_575.CEL.gz,0.795895,0.096750,0.000000,0.0,0.0,0.0,0.000000,0.039187,0.068168,0.0
GSM1587834_FL_lymph_node_biopsy_untreated_581.CEL.gz,0.469749,0.309251,0.000000,0.0,0.0,0.0,0.187484,0.033515,0.000000,0.0
GSM1587835_FL_lymph_node_biopsy_untreated_598.CEL.gz,0.601933,0.112202,0.161871,0.0,0.0,0.0,0.102334,0.021659,0.000000,0.0
GSM1587836_FL_lymph_node_biopsy_untreated_639.CEL.gz,0.493989,0.126790,0.065926,0.0,0.0,0.0,0.262884,0.050411,0.000000,0.0
GSM1587837_FL_lymph_node_biopsy_untreated_664.CEL.gz,0.709893,0.145104,0.000000,0.0,0.0,0.0,0.058381,0.059191,0.027429,0.0
GSM1587838_FL_lymph_node_biopsy_untreated_666.CEL.gz,0.884723,0.055305,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.059972,0.0
GSM1587839_FL_lymph_node_biopsy_untreated_695.CEL.gz,0.895648,0.000000,0.000000,0.0,0.0,0.0,0.104352,0.000000,0.000000,0.0
GSM1587840_FL_lymph_node_biopsy_untreated_706.CEL.gz,0.672551,0.153535,0.000000,0.0,0.0,0.0,0.131373,0.021689,0.020851,0.0


In [56]:
absolute = result.Subjects[0].MIXabs[0].reset_index().to_dict('records')
proportions = result.Subjects[0].MIXprop[0].reset_index().to_dict('records')
metrics = result.Subjects[0].ACCmetrix[0].reset_index().to_dict('records')
metrics.to_dict('records')

[{'index': 'GSM1587831_FL_lymph_node_biopsy_untreated_1063.CEL.gz',
  'RMSEa': 0.7356467701374008,
  'RMSEp': 1.3580584756979353,
  'Ra': 0.7659924064451571,
  'Rp': 0.7659924064451571,
  'BestParams': 0.25,
  'Iter': 4,
  'IscBySbj': 1.3614559595090132,
  'IscPob': 1.4338058519231907},
 {'index': 'GSM1587832_FL_lymph_node_biopsy_untreated_1080.CEL.gz',
  'RMSEa': 0.7505897249244129,
  'RMSEp': 1.090406026517986,
  'Ra': 0.7110050497112939,
  'Rp': 0.7110050497112943,
  'BestParams': 0.75,
  'Iter': 3,
  'IscBySbj': 1.4812507549846523,
  'IscPob': 1.7360281133525028},
 {'index': 'GSM1587833_FL_lymph_node_biopsy_untreated_575.CEL.gz',
  'RMSEa': 0.732099068814191,
  'RMSEp': 1.2091834335647023,
  'Ra': 0.7288513928726023,
  'Rp': 0.7288513928726023,
  'BestParams': 0.75,
  'Iter': 3,
  'IscBySbj': 1.5709810234715755,
  'IscPob': 1.3941344285023054},
 {'index': 'GSM1587834_FL_lymph_node_biopsy_untreated_581.CEL.gz',
  'RMSEa': 0.7275200560756733,
  'RMSEp': 0.7677020173040355,
  'Ra': 0.

In [47]:
np.mean(result.Subjects[0].ACCmetrix[0].IscBySbj.values)+2*np.std(result.Subjects[0].ACCmetrix[0].IscBySbj.values)

1.7015024184163652

In [6]:
# Getting Real Betas Matrix
vector = [x[0] for x in subject]
columns = ['V' + str(x+1) for x in range(len(subject))]
Betas = pd.DataFrame(np.column_stack(vector), columns=columns)

NameError: name 'subject' is not defined

In [ ]:
def blandAltman (betas, result):
    betahat = result.Subjects[0].MIXprop[0].T.to_numpy(copy=True)
    betahat = betahat.flatten()
    
    betasim = betas.to_numpy(copy=True)
    betasim = betasim.flatten()
    
    df = pd.DataFrame(np.transpose([betahat, betasim, (betahat - betasim)]), columns = ['betahat', 'betasim', 'difs'])
    summary = pd.DataFrame([[np.mean(df.difs),np.std(df.difs),np.min(df.difs),np.max(df.difs)]], columns = ['mn', 'sd', 'mi', 'mx'])
    return df, summary
       

In [ ]:
ba, summary = blandAltman (Betas, result)

In [ ]:
def blandAltamGraph(ba, summary):
    fig, ax = plt.subplots(figsize=(16,8))
    ax.plot(ba.betasim, ba.difs, 'k.')
    plt.xlabel('Beta Sim')
    plt.ylabel('Difs')
    ax.axhline(summary.mn[0])
    ax.axhline(summary.mn[0]+2*summary.sd[0])
    ax.axhline(summary.mn[0]-2*summary.sd[0])

In [ ]:
blandAltamGraph(ba, summary)